# __Download Datasets__

## IMDB Datasets

[Imbd datastets](https://datasets.imdbws.com/)

## TMDB Datasets

[Full TMDB Movies](https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/data)

[TMDB Dataset](https://drive.google.com/file/d/1VB5_gl1fnyBDzcIOXZ5vUSbCY68VZN1v/view?usp=sharing)

# **TRANSFORMING DATA**

## **GENERAL FUNCTIONS**

#### **imports**

In [3]:
from enum import StrEnum, unique, auto
from IPython.display import clear_output
from typing import List, Dict
import csv
import re

#### **Clean Value function**

In [5]:
def clean_value(value, value_type=str):
    """Cleans the value according to the specified data type."""
    if value is None:
        return None
    value = str(value).replace("\\N", "").strip('"\n\t\' ')
    
    if value_type == str:
        if value == "":
            value = None
        else:
            value = value.lower()
    elif value_type == int:
        try:
            value = int(value)
        except:
            value = None
    elif value_type == bool:
        try:
            value = int(value)
            value = value if value in [0,1] else None
        except:
            value_tmp = value.lower()
            value = int(value_tmp == "true") if value_tmp in ["false", "true"] else None
    elif value_type == float:
        try:
            value = float(value)
        except:
            value = None
    elif value_type == list:
        value = re.sub('[\'"\\[\\]]', "", value)
        if isinstance(value, str) and value.count(",") > 0:
            value = value.split(",")
        elif isinstance(value, str) and len(value) > 0:
            value = [value]
        elif not isinstance(value, list):
            value = []
   
    return value

#### **Extracting unique values function**

In [7]:
def extract_unique_column_values(file_path, delimiter, column_name_in_file, column_type=str, type_if_list=str, encoding='utf-8'):

        unique_values = set()

        try:
            with open(file_path, newline='', encoding=encoding) as file:
                reader = csv.DictReader(file, delimiter=delimiter)

                for row in reader:
                    value = row[column_name_in_file]
                    cleaned_value = clean_value(value, value_type=column_type)
                    if isinstance(cleaned_value, list):
                        for item in cleaned_value:
                            cleaned_item = clean_value(item, value_type=type_if_list)
                            if cleaned_item is not None:
                                unique_values.add(cleaned_item)
                    elif cleaned_value is not None:
                        unique_values.add(cleaned_value)

        except FileNotFoundError as e:
            print(f"Error: File not found: {e}")
            raise
        except csv.Error as e:
            print(f"Error reading the file: {e}")
            raise

        return unique_values

#### **Extracting uniques tuples values function**

In [9]:
def extract_unique_tuples_values(file_path, delimiter, columns: Dict, encoding='utf-8'):

        unique_values = set()

        try:
            with open(file_path, newline='', encoding=encoding) as file:
                reader = csv.DictReader(file, delimiter=delimiter)

                for row in reader:
                    tuple_value = tuple([clean_value(row[column],type_value) for column, type_value in columns.items()])
                    unique_values.add(tuple_value)

        except FileNotFoundError as e:
            print(f"Error: File not found: {e}")
            raise
        except csv.Error as e:
            print(f"Error reading the file: {e}")
            raise

        return [{key:value for key, value in zip(columns.keys(),tuple_n)} for tuple_n in unique_values]

#### **Printing unique values function**

In [11]:
def print_unique_values(unique_values, name, columns=3):
    print(f"TOTAL : {len(unique_values)}")
    print(f"{name.upper()} :")
    max_length = max(len(value) for value in unique_values) + 4
    
    for index, value in enumerate(unique_values, start=1):
        if index % columns == 1 and columns > 1:
            print(f"\t{value.ljust(max_length)}", end="")
        elif columns == 1 or (index % columns) == 0:
            print(f"{value}")
        else:
            print(f"{value.ljust(max_length)}", end="")

#### **Cartesien product of dictionnaries**

In [13]:
def get_cartesian_product(pools:List[List[Dict]]):
    result = [{}]
    for pool in pools:
        result = [{**x, **y} for x in result for y in pool]
    return result

#### **Cartesien product of two listes**

In [15]:
def get_cartesian_product_simple(list_1: List, list_2: List):
    result = [(x, y) for x in list_1 for y in list_2]
    return result

## **INITIAL ANALYSE**

### **IMDB FILE: `title.basics.tsv`**

In [22]:
file_path = "../Data_Files/data/title.basics.tsv"
delimiter = "\t"

#### **Genres**

In [263]:
unique_genres = sorted(extract_unique_column_values(file_path, delimiter, "genres", column_type=list))
print_unique_values(unique_genres, "genres", 4)

TOTAL : 28
GENRES :
	Action         Adult          Adventure      Animation
	Biography      Comedy         Crime          Documentary
	Drama          Ews            Family         Fantasy
	Film-noir      Game-show      History        Horror
	Music          Musical        Mystery        Reality-tv
	Romance        Sci-fi         Short          Sport
	Talk-show      Thriller       War            Western


#### **Formats**

In [161]:
unique_formats = sorted(extract_unique_column_values(file_path, delimiter, "titleType", column_type=str))
print_unique_values(unique_formats, "Format", 4)

TOTAL : 11
FORMAT :
	Movie           Short           Tvepisode       Tvminiseries
	Tvmovie         Tvpilot         Tvseries        Tvshort
	Tvspecial       Video           Videogame       

### **IMDB FILE: `title.akas.tsv`**

In [279]:
file_path = "../Data_Files/data/title.akas.tsv"
delimiter = "\t"

#### **Types**

In [281]:
unique_types = sorted(extract_unique_column_values(file_path, delimiter, "types", column_type=list))
print_unique_values(unique_types, "Types", 4)

TOTAL : 23
TYPES :
	Alternative             Alternativefestival    Alternativetv          Alternativevideo
	Dvd                     Dvdalternative         Dvdvideo               Festival
	Imdbdisplay             Imdbdisplaydvd         Imdbdisplayfestival    Imdbdisplaytv
	Imdbdisplayvideo       Imdbdisplayworking     Original                Tv
	Tvvideo                Video                   Working                 Workingalternative
	Workingfestival        Workingtv              Workingvideo           

In [283]:
types_imdb = ["alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay"]

#### **Languages**

In [202]:
unique_languages = sorted(extract_unique_column_values(file_path, delimiter, "language", column_type=str))
print_unique_values(unique_languages, "Languages", 12)

TOTAL : 108
LANGUAGES :
	Af     Am     Ar     Az     Be     Bg     Bn     Br     Bs     Ca     Cmn    Cr
	Cs     Cy     Da     De     Eka    El     En     Es     Et     Eu     Fa     Fi
	Fr     Fro    Ga     Gd     Gl     Gsw    Gu     Haw    He     Hi     Hil    Hr
	Hu     Hy     Id     Is     It     Iu     Ja     Jsl    Jv     Ka     Kk     Kn
	Ko     Ku     Ky     La     Lb     Lo     Lt     Lv     Mi     Mk     Ml     Mn
	Mr     Ms     My     Myv    Ne     Nl     No     Pa     Pl     Prs    Ps     Pt
	Qac    Qal    Qbn    Qbo    Qbp    Rm     Rn     Ro     Roa    Ru     Sd     Sk
	Sl     Sq     Sr     St     Su     Sv     Ta     Te     Tg     Th     Tk     Tl
	Tn     Tr     Uk     Ur     Uz     Vi     Wo     Xh     Yi     Yue    Zh     Zu


#### **Regions**

In [206]:
unique_regions = sorted(extract_unique_column_values(file_path, delimiter, "region", column_type=str))
print_unique_values(unique_regions, "Regions", 12)

TOTAL : 244
REGIONS :
	A       Ad      Ae      Af      Ag      Ai      Al      Am      Ao      Aq      Ar      As
	At      Au      Aw      Az      B       Ba      Bb      Bd      Be      Bf      Bg      Bh
	Bi      Bj      Bm      Bo      Br      Bs      Bt      Bumm    Bw      By      Bz      C
	Ca      Cc      Cd      Cf      Cg      Ch      Ci      Ck      Cl      Cm      Co      Cr
	Cshh    Csxx    Cu      Cv      Cw      Cy      Cz      Ddde    De      Dj      Dk      Dm
	Do      Dz      E       Ec      Ee      Eg      Eh      Er      Es      Et      Fi      Fj
	Fm      Fo      Fr      G       Ga      Gb      Gd      Ge      Gf      Gh      Gi      Gl
	Gm      Gp      Gq      Gr      Gt      Gu      Gw      Gy      H       Hk      Hr      Ht
	Hu      I       Id      Ie      Il      Im      Iq      Ir      Is      It      Je      Jm
	Jo      Jp      K       Ke      Kg      Kh      Ki      Km      Kp      Kr      Kw      Ky
	Kz      L       La      Lb      Lc      Li      Lk      Lr

### **IMDB FILE: `title.principal.tsv`**

In [9]:
file_path = "../Data_Files/data/title.principals.tsv"
delimiter = "\t"

#### **Job Titles**

It's not a interesting field because there are 42663 jobs, so, there are not generate a relation.

#### **Job Categories**

In [193]:
unique_jobs_categories = sorted(extract_unique_column_values(file_path, delimiter, "category", column_type=str))
print_unique_values(unique_jobs_categories, "Jobs Categories", 4)

TOTAL : 13
JOBS CATEGORIES :
	Actor                  Actress                Archive_footage        Archive_sound
	Casting_director       Cinematographer        Composer               Director
	Editor                 Producer               Production_designer    Self
	Writer                 

#### **Creating new file with only tuples (tconst, nconst, category)**

In [ ]:
with open("../Data_Files/data/title.principals.filtered.tsv", "w", newline="", encoding="utf-8") as new_file:
    columns = {'tconst': str, "nconst": str, "category": str}
    writer = csv.DictWriter(new_file,fieldnames=columns.keys(), delimiter=delimiter)
    writer.writeheader()
    uniques = extract_unique_tuples_values(file_path=file_path, delimiter=delimiter, columns=columns)

### **IMDB FILE: `name.basics.tsv`**

In [209]:
file_path = "../Data_Files/data/name.basics.tsv"
delimiter = "\t"

#### **Professions**

In [211]:
unique_professions = sorted(extract_unique_column_values(file_path, delimiter, "primaryProfession", column_type=list))
print_unique_values(unique_professions, "Professions", 4)

TOTAL : 46
PROFESSIONS :
	Accountant                   Actor                        Actress                      Animation_department
	Archive_footage              Archive_sound                Art_department               Art_director
	Assistant                    Assistant_director           Camera_department            Casting_department
	Casting_director             Choreographer                Cinematographer              Composer
	Costume_department           Costume_designer             Director                     Editor
	Editorial_department         Electrical_department        Executive                    Legal
	Location_management          Make_up_department           Manager                      Miscellaneous
	Music_artist                 Music_department             Podcaster                    Producer
	Production_department        Production_designer          Production_manager           Publicist
	Script_department            Set_decorator                Sound_department

In [326]:
count = 1
for job in unique_jobs_categories:
    if job not in unique_professions:
        print(job)

Self


### **TMDB FILE : `tmdb_full_separator.csv`** 

In [49]:
file_path = "../Data_Files/data/TMDB_movie_dataset_v11_separator.csv"
delimiter = "|"

#### **Production Countries**

In [269]:
unique_production_countries = sorted(extract_unique_column_values(file_path, delimiter, "production_countries", column_type=list))
print_unique_values(unique_production_countries, "Production Countries", 12)

TOTAL : 228
PRODUCTION COUNTRIES :
	A     Ad    Ae    Af    Ag    Ai    Al    Am    Ao    Aq    Ar    At
	Au    Aw    Az    B     Ba    Bb    Bd    Be    Bf    Bg    Bh    Bi
	Bj    Bm    Bo    Br    Bs    Bt    Bw    By    Bz    C     Ca    Cd
	Cf    Cg    Ch    Ci    Ck    Cl    Cm    Co    Cr    Cs    Cu    Cv
	Cx    Cy    Cz    De    Dj    Dk    Dm    Do    Dz    E     Ec    Ee
	Eg    Eh    Er    Es    Et    F     Fi    Fj    Fk    Fm    Fo    Fr
	G     Ga    Gb    Gd    Ge    Gf    Gh    Gi    Gl    Gm    Gp    Gq
	Gr    Gt    Gu    Gw    Gy    H     Hk    Hr    Ht    Hu    I     Id
	Ie    Il    Io    Iq    Ir    Is    It    Jm    Jo    Jp    K     Ke
	Kg    Kh    Ki    Km    Kp    Kr    Kw    Ky    Kz    L     La    Lb
	Lc    Li    Lk    Lr    Ls    Lt    Lu    Lv    Ly    M     Ma    Mc
	Md    Me    Mg    Mh    Mk    Ml    Mm    Mo    Mq    Mr    Ms    Mt
	Mu    Mv    Mw    Mx    My    Mz    O     Om    P     Pa    Pe    Pf
	Pg    Ph    Pk    Pl    Pm    Pr    Ps    Pt    Pw    

In [51]:
unique_imdb_ids = sorted(extract_unique_column_values(file_path, delimiter, "imdb_id", column_type=str))
print(len(unique_imdb_ids))

606752


In [53]:
unique_tmdb_ids = sorted(extract_unique_column_values(file_path, delimiter, "id", column_type=str))
print(len(unique_tmdb_ids))

1158058


#### **Original languages**

In [273]:
unique_original_languages = sorted(extract_unique_column_values(file_path, delimiter, "original_language", column_type=str))
print_unique_values(unique_original_languages, "Original Languages", 12)

TOTAL : 152
ORIGINAL LANGUAGES :
	Ab    Af    Ak    Am    Ar    As    Ay    Az    Ba    Be    Bg    Bm
	Bn    Bo    Bs    Ca    Ce    Cn    Co    Cr    Cs    Cv    Cy    Da
	De    Dv    Dz    El    En    Eo    Es    Et    Eu    Fa    Ff    Fi
	Fo    Fr    Fy    Ga    Gd    Gl    Gn    Gu    Ha    He    Hi    Hr
	Ht    Hu    Hy    Ia    Id    Ie    Ig    Ik    Is    It    Iu    Ja
	Jv    Ka    Kg    Kk    Kl    Km    Kn    Ko    Ks    Ku    Kw    Ky
	La    Lb    Lg    Li    Ln    Lo    Lt    Lv    Mg    Mh    Mi    Mk
	Ml    Mn    Mo    Mr    Ms    Mt    My    Nb    Ne    Nl    Nn    No
	Nv    Ny    Oc    Om    Or    Os    Pa    Pl    Ps    Pt    Qu    Rm
	Ro    Ru    Rw    Sa    Sc    Sd    Se    Sg    Sh    Si    Sk    Sl
	Sm    So    Sq    Sr    Ss    St    Su    Sv    Sw    Ta    Te    Tg
	Th    Tk    Tl    Tn    To    Tr    Tt    Tw    Ug    Uk    Ur    Uz
	Vi    Wo    Xh    Xx    Yi    Yo    Zh    Zu    

#### **Spoken languages**

In [275]:
unique_spoken_languages = sorted(extract_unique_column_values(file_path, delimiter, "spoken_languages", column_type=list))
print_unique_values(unique_spoken_languages, "Spoken Languages", 12)

TOTAL : 177
SPOKEN LANGUAGES :
	Aa    Ab    Af    Ak    Am    An    Ar    As    Av    Ay    Az    Ba
	Be    Bg    Bi    Bm    Bn    Bo    Br    Bs    Ca    Ce    Ch    Cn
	Co    Cr    Cs    Cu    Cv    Cy    Da    De    Dv    Dz    Ee    El
	En    Eo    Es    Et    Eu    Fa    Ff    Fi    Fj    Fo    Fr    Fy
	Ga    Gd    Gl    Gn    Gu    Ha    He    Hi    Ho    Hr    Ht    Hu
	Hy    Hz    Ia    Id    Ie    Ig    Ii    Ik    Io    Is    It    Iu
	Ja    Jv    Ka    Kg    Ki    Kk    Kl    Km    Kn    Ko    Ks    Ku
	Kw    Ky    La    Lb    Lg    Li    Ln    Lo    Lt    Lv    Mg    Mh
	Mi    Mk    Ml    Mn    Mo    Mr    Ms    Mt    My    Na    Nb    Ne
	Ng    Nl    Nn    No    Nr    Nv    Ny    Oc    Oj    Om    Or    Os
	Pa    Pi    Pl    Ps    Pt    Qu    Rm    Rn    Ro    Ru    Rw    Sa
	Sc    Sd    Se    Sg    Sh    Si    Sk    Sl    Sm    Sn    So    Sq
	Sr    Ss    St    Su    Sv    Sw    Ta    Te    Tg    Th    Ti    Tk
	Tl    Tn    To    Tr    Tt    Tw    Ty    Ug    Uk    Ur  

#### **Status**

In [277]:
unique_status = sorted(extract_unique_column_values(file_path, delimiter, "status", column_type=list))
print_unique_values(unique_status, "Status", 8)

TOTAL : 6
STATUS :
	Canceled           In production      Planned            Post production    Released           Rumored            

#### **Production Companies**

In [317]:
unique_production_companies = sorted(extract_unique_column_values(file_path, delimiter, "production_companies_name", column_type=list))
print_unique_values(unique_production_companies[:100], "Production Companies", 2)

TOTAL : 100
PRODUCTION COMPANIES :
	20th century fox                            20th century fox animation
	a band apart                                albert s. ruddy productions
	altavista films                             amblin entertainment
	amercent films                              american entertainment partners l.p.
	american zoetrope                           angel films
	anonymous content                           ard
	arte                                        arte france cinéma
	arts council of england                     atlas entertainment
	atresmedia                                  augustus film
	bakshi productions                          bbc film
	beijing new picture film co. ltd.           benderspink
	beta film                                   bill/phillips
	blind spot pictures                         bruce brown films
	buena vista international                   c & l
	canal+                                      castle rock entertainment
	catfish productions   

#### **Production Companies Countries**

In [321]:
unique_production_companies_countries = sorted(extract_unique_column_values(file_path, delimiter, "production_companies_country", column_type=list))
print_unique_values(unique_production_companies_countries, "Production Companies Countries", 10)

TOTAL : 25
PRODUCTION COMPANIES COUNTRIES :
	at    ch    de    dk    es    fi    fr    gb    hk    il
	is    it    jp    kr    l     ma    mt    mx    pl    ps
	py    se    us    uy    z     

### **Languages and regions/countries verifications** 

#### **Languages codes to use**

In [359]:
with open("../Data_Files/data/language-codes.csv") as file:
    reader = csv.DictReader(file, delimiter=",")
    languages_in_file = set()
    for row in reader:
        languages_in_file.add(row['code'].lower())
    spoken_languages = set([language.lower() for language in unique_spoken_languages])
    original_languages = set([language.lower() for language in unique_original_languages])
    imdb_languages = set([language.lower() for language in unique_languages])
    
    spoken_languages_not_in_file = spoken_languages.difference(languages_in_file)
    original_languages_not_in_file = original_languages.difference(languages_in_file)
    imdb_languages_not_in_file = imdb_languages.difference(languages_in_file)

    print("LANGUAGES NOT FOUND :")
    print(spoken_languages_not_in_file)
    print(original_languages_not_in_file)
    print(imdb_languages_not_in_file)

LANGUAGES NOT FOUND :
set()
set()
{'qac'}


#### **Regions codes to use**

In [357]:
with open("../Data_Files/data/regions-codes.csv") as file:
    reader = csv.DictReader(file, delimiter=",")
    regions_in_file = set()
    for row in reader:
        regions_in_file.add(row['code'].lower())
    production_countries = set([region.lower() for region in unique_production_countries])
    production_companies_countries = set([region.lower() for region in unique_production_companies_countries])
    imdb_regions = set([region.lower() for region in unique_regions])
    
    production_countries_not_in_file = production_countries.difference(regions_in_file)
    production_companies_countries_not_in_file = production_companies_countries.difference(regions_in_file)
    imdb_regions_not_in_file = imdb_regions.difference(regions_in_file)

    print("REGIONS NOT FOUND :")
    print(production_countries_not_in_file)
    print(production_companies_countries_not_in_file)
    print(imdb_regions_not_in_file)

REGIONS NOT FOUND :
{'z', 'h', 'xg', 'm', 't', 's', 'g', 'p', 'i', 'e', 'o', 'a', 'c', 'b', 'f', 'v', 'k', 'l', 'xi', 'xc'}
{'z', 'l'}
{'z', 's', 'c', 'h', 'e', 'b', 'g', 'v', 'k', 'p', 'l', 'i', 'u', 'o', 'r', 'm', 't', 'a'}


### **Creation of file `productions_companies.csv` from `tmdb_full_separator.csv` file** 

In [86]:
production_companies_file = "../Data_Files/data/production_companies.csv"
delimiter_production_companies_file = "|"

In [211]:
def process_str_like_lists(text1, text2):
    list_1 = clean_value(text1, list)
    list_2 = clean_value(text2, list)
    result_list = []
    
    list_1_clean = [clean_value(value, str) for value in list_1]
    list_2_clean = [clean_value(value, str) for value in list_2]

    size_list_1 = len(list_1_clean)
    size_list_2 = len(list_2_clean)
    
    
    if size_list_1 > 0:
        for val in reversed(list_2_clean):
            if val == None : list_2_clean.pop()
            else : break
        size_list_2 = len(list_2_clean)
                    
        if size_list_2 >= size_list_1:
            result_list = [(val_1, val_2) for val_1, val_2 in zip(list_1_clean, list_2_clean)] 
        elif size_list_2 > 0:
            result_list = [(val_1, val_2) for val_1, val_2 in zip(list_1_clean[:size_list_2], list_2_clean)]
            result_list += get_cartesian_product_simple(list_1_clean[size_list_2:],[list_2_clean[-1]])                
        else:
            result_list = [(val_1, None) for val_1 in list_1_clean]
        
    return result_list


In [193]:
def list_of_tuples_to_dict_key_set(list_of_tuples: List) -> Dict:
    result_dict = {}
    for val1, val2 in list_of_tuples:
        if val1 not in result_dict.keys():
            result_dict[val1] = None
        if val2 != None: result_dict[val1] = val2
    return result_dict

In [215]:
companies_in_file = set()

with open("../Data_Files/data/tmdb_full_separator.csv", "r", newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file, delimiter="|")
    print("Reading file...")
    for row in reader:
        production_companies_name = row['production_companies_name']
        production_companies_country = row['production_companies_country']
        
        companies_in_file.update(process_str_like_lists(
            production_companies_name, 
            production_companies_country
        ))
companies_region_dict = list_of_tuples_to_dict_key_set(companies_in_file)           
companies_in_file_dict = [{"company_name": company, "region_code": region} for company, region in companies_region_dict.items() if company is not None]
companies_in_file_dict = sorted(companies_in_file_dict, key=lambda x: x["company_name"])
with open(production_companies_file, "w", newline='', encoding='utf-8') as file:
    print("Writing file...")
    fieldnames = ["company_name", "region_code"]
    writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter=delimiter_production_companies_file)
    writer.writeheader()
    writer.writerows(companies_in_file_dict)
    print("File writed with success...")

Reading file...
Writing file...
File writed with success...


---
---

## **FILES INFORMATION**

### __Script `info_file.sh`__

```bash
#!/bin/bash


separator="\\t"

while getopts 's:f:' OPTION; do 
  case "$OPTION" in 
    s)
      separator="$OPTARG"
      ;;
    f)
      archivo="$OPTARG"
      ;;
  esac
done


if [ -z "$archivo" ]; then
  echo "Uso: $0 -f <file>"
  exit 1
fi


filename=$(basename -- "$archivo")
filename_output="${filename%.*}_info.txt"

echo -e "FILENAME: $filename \nNB LINES: $(wc -l < $archivo) \nCOLUMNS (name/max-longuer) :" > $filename_output

awk -F$separator '
NR == 1 { 
    # Save columns names in an array
    for(i=1; i<=NF; i++) {
        nombres[i] = $i
        max_len[i] = 0
    }
}
NR > 1 { 
    # Calculate the longuer max of each column
    for(i=1; i<=NF; i++) {
        if($i != "\\N") { 
            len = length($i)
            if(len > max_len[i]) max_len[i] = len
        }
    }
}
END {
    # Print names columns and longuer max
    for(i=1; i<=length(nombres); i++) {
        print "  "nombres[i] "\t" max_len[i]
    }
}
' "$archivo" >> "${filename_output}"
```

### **Using the script bash `info_file.sh`**

```sh
$./info_file -s "<separator>" f <file_path> 
```

## **CREATION BDD**

In [23]:
import sqlite3 as sql


sqlite3_db = "../Database/movies_recommendation.db"

### **MCD**

![mcd movies recommendation](https://imgur.com/ebnbvQi.png)

### **MLD**

![mld movies recommendation](https://imgur.com/Rkwz1nB.png)

### **SCRIPT SQLITE**

In [1]:
sqlite3_script = """
------------------------------------------------------------
--        Script SQLite  
------------------------------------------------------------


------------------------------------------------------------
-- Table: Region
------------------------------------------------------------
CREATE TABLE Region(
	region_id      INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT ,
	region_name    TEXT NOT NULL ,
	region_code    TEXT NOT NULL
);


------------------------------------------------------------
-- Table: Language
------------------------------------------------------------
CREATE TABLE Language(
	language_id      INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT ,
	language_name    TEXT NOT NULL ,
	language_code    TEXT NOT NULL
);


------------------------------------------------------------
-- Table: Type
------------------------------------------------------------
CREATE TABLE Type(
	type_id      INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT ,
	type_name    TEXT NOT NULL
);


------------------------------------------------------------
-- Table: Genre
------------------------------------------------------------
CREATE TABLE Genre(
	genre_id      INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT ,
	genre_name    TEXT NOT NULL
);


------------------------------------------------------------
-- Table: Person
------------------------------------------------------------
CREATE TABLE Person(
	person_Id       TEXT NOT NULL ,
	primary_name    TEXT ,
	birth_year      INTEGER ,
	death_year      INTEGER,
	CONSTRAINT Person_PK PRIMARY KEY (person_Id)
);


------------------------------------------------------------
-- Table: Profession
------------------------------------------------------------
CREATE TABLE Profession(
	profession_id      INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT ,
	profession_name    TEXT NOT NULL
);


------------------------------------------------------------
-- Table: Production_company
------------------------------------------------------------
CREATE TABLE Production_company(
	production_company_id      INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT ,
	production_company_name    TEXT NOT NULL ,
	region_id                  INTEGER NOT NULL

	,CONSTRAINT Production_company_Region_FK FOREIGN KEY (region_id) REFERENCES Region(region_id)
);


------------------------------------------------------------
-- Table: Format
------------------------------------------------------------
CREATE TABLE Format(
	format_id      INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT ,
	format_name    TEXT NOT NULL
);


------------------------------------------------------------
-- Table: Title
------------------------------------------------------------
CREATE TABLE Title(
	title_id          TEXT NOT NULL ,
	original_title    TEXT,
	primary_title     TEXT,
	is_adult          INTEGER ,
	start_year        INTEGER ,
	end_year          INTEGER ,
	budget            INTEGER ,
	revenue           INTEGER ,
	overview          TEXT,
	runtime           INTEGER,
	format_id         INTEGER,
	CONSTRAINT Title_PK PRIMARY KEY (title_id)

	,CONSTRAINT Title_Format_FK FOREIGN KEY (format_id) REFERENCES Format(format_id)
);


------------------------------------------------------------
-- Table: Episode
------------------------------------------------------------
CREATE TABLE Episode(
	episode_id        TEXT NOT NULL ,
	season_number     INTEGER ,
	episode_number    INTEGER ,
	title_id          TEXT NOT NULL,
	CONSTRAINT Episode_PK PRIMARY KEY (episode_id)

	,CONSTRAINT Episode_Title_FK FOREIGN KEY (title_id) REFERENCES Title(title_id)
);


------------------------------------------------------------
-- Table: Regional_Title
------------------------------------------------------------
CREATE TABLE Regional_Title(
	regional_title_id    INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT ,
	regional_title       TEXT,
	is_original_title    INTEGER ,
	region_id            INTEGER ,
	language_id          INTEGER ,
	title_id             TEXT NOT NULL

	,CONSTRAINT Regional_Title_Region_FK FOREIGN KEY (region_id) REFERENCES Region(region_id)
	,CONSTRAINT Regional_Title_Language0_FK FOREIGN KEY (language_id) REFERENCES Language(language_id)
	,CONSTRAINT Regional_Title_Title1_FK FOREIGN KEY (title_id) REFERENCES Title(title_id)
);


------------------------------------------------------------
-- Table: Rating
------------------------------------------------------------
CREATE TABLE Rating(
	rating_id         INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT ,
	average_rating    REAL ,
	num_votes         INTEGER ,
	title_id          TEXT NOT NULL

	,CONSTRAINT Rating_Title_FK FOREIGN KEY (title_id) REFERENCES Title(title_id)
);


------------------------------------------------------------
-- Table: Types_of_title
------------------------------------------------------------
CREATE TABLE Types_of_title(
	type_id     INTEGER NOT NULL ,
	title_id    TEXT NOT NULL,
	CONSTRAINT Types_of_title_PK PRIMARY KEY (type_id,title_id)

	,CONSTRAINT Types_of_title_Type_FK FOREIGN KEY (type_id) REFERENCES Type(type_id)
	,CONSTRAINT Types_of_title_Title0_FK FOREIGN KEY (title_id) REFERENCES Title(title_id)
);


------------------------------------------------------------
-- Table: Genres_of_title
------------------------------------------------------------
CREATE TABLE Genres_of_title(
	title_id    TEXT NOT NULL,
	genre_id    INTEGER NOT NULL,
	CONSTRAINT Genres_of_title_PK PRIMARY KEY (title_id,genre_id)

	,CONSTRAINT Genres_of_title_Title_FK FOREIGN KEY (title_id) REFERENCES Title(title_id)
	,CONSTRAINT Genres_of_title_Genre0_FK FOREIGN KEY (genre_id) REFERENCES Genre(genre_id)
);


------------------------------------------------------------
-- Table: Crew
------------------------------------------------------------
CREATE TABLE Crew(
    crew_id           INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	title_id          TEXT NOT NULL ,
	person_Id         TEXT NOT NULL ,
	profession_id     INTEGER NOT NULL

	,CONSTRAINT Crew_Title_FK FOREIGN KEY (title_id) REFERENCES Title(title_id)
	,CONSTRAINT Crew_Person0_FK FOREIGN KEY (person_Id) REFERENCES Person(person_Id)
	,CONSTRAINT Crew_Profession1_FK FOREIGN KEY (profession_id) REFERENCES Profession(profession_id)
);


------------------------------------------------------------
-- Table: Title_by_production_company
------------------------------------------------------------
CREATE TABLE Title_by_production_company(
	title_id                 TEXT NOT NULL ,
	production_company_id    INTEGER NOT NULL,
	CONSTRAINT Title_by_production_company_PK PRIMARY KEY (title_id,production_company_id)

	,CONSTRAINT Title_by_production_company_Title_FK FOREIGN KEY (title_id) REFERENCES Title(title_id)
	,CONSTRAINT Title_by_production_company_Production_company0_FK FOREIGN KEY (production_company_id) REFERENCES Production_company(production_company_id)
);


------------------------------------------------------------
-- Table: Spoken_languages_in_title
------------------------------------------------------------
CREATE TABLE Spoken_languages_in_title(
	title_id       TEXT NOT NULL ,
	language_id    INTEGER NOT NULL,
	CONSTRAINT Spoken_languages_in_title_PK PRIMARY KEY (title_id,language_id)

	,CONSTRAINT Spoken_languages_in_title_Title_FK FOREIGN KEY (title_id) REFERENCES Title(title_id)
	,CONSTRAINT Spoken_languages_in_title_Language0_FK FOREIGN KEY (language_id) REFERENCES Language(language_id)
);


------------------------------------------------------------
-- Table: Title_production_region
------------------------------------------------------------
CREATE TABLE Title_production_region(
	region_id    INTEGER NOT NULL ,
	title_id     TEXT NOT NULL,
	CONSTRAINT Title_production_region_PK PRIMARY KEY (region_id,title_id)

	,CONSTRAINT Title_production_region_Region_FK FOREIGN KEY (region_id) REFERENCES Region(region_id)
	,CONSTRAINT Title_production_region_Title0_FK FOREIGN KEY (title_id) REFERENCES Title(title_id)
);


------------------------------------------------------------
-- Table: Top_3_Professions
------------------------------------------------------------
CREATE TABLE Top_3_Professions(
	profession_id    INTEGER NOT NULL ,
	person_Id        TEXT NOT NULL,
	CONSTRAINT Top_3_Professions_PK PRIMARY KEY (profession_id,person_Id)

	,CONSTRAINT Top_3_Professions_Profession_FK FOREIGN KEY (profession_id) REFERENCES Profession(profession_id)
	,CONSTRAINT Top_3_Professions_Person0_FK FOREIGN KEY (person_Id) REFERENCES Person(person_Id)
);
"""

### **SCRIPT MYSQL**

### **CREATION SQLITE DATABASE**

In [92]:
try:
    conn = sql.connect(sqlite3_db)
    conn.executescript(sqlite3_script)
    print("DBB Created")
except Exception as exception:
    print(exception)
finally:
    conn.close()

DBB Created


## **CODING : FROM FILE TO SQL**

In [25]:
import sqlite3
import csv
import mysql.connector

csv.field_size_limit(100000000)

131072

#### **`TypeOfTable` Enum**

In [27]:
@unique
class TypeOfTable(StrEnum):
    REGULAR = auto()
    FLAT = auto()
    SIMPLE_JOIN = auto()
    MULTI_JOIN = auto()

#### **`SQLiteDatabase` Class**

In [29]:
# SQLite database handler class
class SQLiteDatabase:
    def __init__(self, db_name):
        self.db_name = db_name
        self.connection = self._create_connection()

    def _create_connection(self):
        """Establishes a connection to the SQLite database."""
        try:
            conn = sqlite3.connect(self.db_name)
            return conn
        except sqlite3.Error as e:
            print(f"Error connecting to SQLite database: {e}")
            return None

    def insert_records(self, chunk, columns, table_name):
        """Inserts records into the SQLite database in bulk."""
        try:
            cursor = self.connection.cursor()
            placeholders = ', '.join([f":{col}" for col in columns])
            query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"
            cursor.executemany(query, chunk)
            self.connection.commit()
            print(f"Inserted records into SQLite table '{table_name}'.")
        except sqlite3.Error as e:
            print(f"Error inserting data into SQLite database: {e}")
            self.connection.rollback()
    
    def update_records(self, chunk, table_name, condition_columns, columns_to_update, condition_operator: str="AND"):
        """Updates records in the SQLite database in bulk using prepared statements."""
        try:
            cursor = self.connection.cursor()
            condition_operator = condition_operator if condition_operator in ["AND", "OR"] else ""
            set_clause = ', '.join([f"{col} = :{col}" for col in columns_to_update])
            condition_columns_placeholder = f" {condition_operator} ".join(
                f"{condition_column} = :{condition_column}"
                for condition_column in condition_columns
            )
            query = f"UPDATE {table_name} SET {set_clause} WHERE {condition_columns_placeholder}"
            cursor.executemany(query, chunk)
            self.connection.commit()
            print(f"Updated records in {table_name} with condition on '{', '.join(condition_columns)}'.")
            
        except sqlite3.Error as e:
            print(f"Error updating records in SQLite database: {e}")
            self.connection.rollback()

    def fetch_records(self, table_name, columns: List = [], condition=None):
        """Fetches records from a table."""
        try:
            cursor = self.connection.cursor()
            columns = ",".join(columns) if len(columns) > 0 else "*"
            query = f"SELECT {columns} FROM {table_name}"
            if condition:
                query += f" WHERE {condition}"
            cursor.execute(query)
            records = cursor.fetchall()
            return records
        except sqlite3.Error as e:
            print(f"Error fetching data from SQLite database: {e}")
            return []
    def fetch_one_record(self, table_name, columns: List = [], conditions: Dict = {}, condition_operator: str="AND"):
        """Fetches records from a table."""
        try:
            cursor = self.connection.cursor()
            condition_operator = condition_operator if condition_operator in ["AND", "OR"] else ""
            condition_placeholder = f" {condition_operator} ".join(
                f"{condition_column} = :{condition_column}"
                for condition_column in conditions.keys()
            )
            columns = ",".join(columns) if len(columns) > 0 else "*"
            query = f"SELECT {columns} FROM {table_name}"
            if len(condition_placeholder) > 0:
                query += f" WHERE {condition_placeholder}"

            cursor.execute(query, conditions)
            record = cursor.fetchone()
            return record
            
        except sqlite3.Error as e:
            print(f"Error fetching data from SQLite database: {e}")
            return None
    
    def close(self):
        """Closes the connection to the SQLite database."""
        if self.connection:
            self.connection.close()
            print("SQLite database connection closed.")

#### **`MySQLDatabase` Class**

In [31]:
# MySQL database handler class
class MySQLDatabase:
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.connection = self._create_connection()

    def _create_connection(self):
        """Establishes a connection to the MySQL database."""
        try:
            conn = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            return conn
        except mysql.connector.Error as e:
            print(f"Error connecting to MySQL database: {e}")
            return None

    def insert_records(self, table_name, columns, data):
        """Inserts records into the MySQL database in bulk."""
        try:
            cursor = self.connection.cursor()
            placeholders = ', '.join(['%s'] * len(columns))  # Parametrization for MySQL
            query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"
            cursor.executemany(query, data)
            self.connection.commit()
            print(f"Inserted records into MySQL table '{table_name}'.")
        except mysql.connector.Error as e:
            print(f"Error inserting data into MySQL database: {e}")
            self.connection.rollback()

    def close(self):
        """Closes the connection to the MySQL database."""
        if self.connection:
            self.connection.close()
            print("MySQL database connection closed.")

#### **`FileProcessor` Class**

In [33]:
class FileProcessor:
    def __init__(self, file_path, delimiter):
        self.file_path = file_path
        self.delimiter = delimiter  # The specified delimiter

    def read_in_chunks(self, columns, referenced_columns = [], rows_with_list = False, chunk_size=10000):
        """Reads the file in chunks and returns the cleaned data."""
        try:
            with open(self.file_path, newline='', encoding='utf-8') as f:
                reader = csv.DictReader(f, delimiter=self.delimiter)
                chunk = []
                size = len(columns) + len(referenced_columns)
                for row in reader:
                    row_to_dict = {}
                    row_to_dict.update(self.__row_to_dict(row, columns))
                    row_to_dict.update(self.__row_to_dict(row, referenced_columns, False))
                    if rows_with_list:
                        row_list = [[{key:value}] for key, value in row_to_dict.items() if not isinstance(value, list) and value is not None]
                        row_list += [[{key:value} for value in values if value is not None] for key, values in row_to_dict.items() if isinstance(values, list)]
                        if len(row_list) == size:
                            row_list = get_cartesian_product(row_list)
                            chunk.extend(row_list)
                    else:
                        chunk.append(row_to_dict)
                    if len(chunk) >= chunk_size:
                        yield chunk
                        chunk = []
                if chunk:
                    yield chunk
        except FileNotFoundError as e:
            print(f"Error: File not found: {e}")
            raise
        except csv.Error as e:
            print(f"Error reading the file: {e}")
            raise

    def __row_to_dict(self, row, columns, are_regular: bool=True, column_types: List = [str, int, float, bool, list]) -> Dict:

        row_like_dict = {}
        if are_regular:
            row_like_dict = {
                col["column_in_table"]: clean_value(row[col["column_in_file"]], value_type=col["column_type"])
                for col in columns
                if col["column_type"] in column_types
            }
        else:
            for col in columns:
                key = col.get("column_in_final_table", None)
                default_value = col.get("default_value", None)
                value_in_file = None
                value = None
                referenced_values = col.get("column_values", [])
                
                if key:
                    if default_value:
                        value = referenced_values.get(default_value, None)
                    else:
                        if col["column_type"] in column_types:
                            value_in_file = clean_value(row[col["column_in_file"]], value_type=col["column_type"])
        
                        if isinstance(value_in_file, list) :
                            value_list = []
                            for val in value_in_file:
                                val = clean_value(val, value_type=col.get('type_if_list',str))
                                value_list.append(referenced_values.get(val, None))
                            value = value_list    
                        elif value_in_file:
                            value = referenced_values.get(value_in_file, None)
                    
                    row_like_dict[key] = value
        return row_like_dict

        
    
    def extract_unique_column_values(self, column_info):
        """Extracts and cleans unique values from a column in the file."""
        column_name_in_file = column_info["column_in_file"]
        db_column_name = column_info["column_in_table"]
        data_type = column_info["column_type"]
        unique_values = set()

        try:
            with open(self.file_path, newline='', encoding='utf-8') as f:
                reader = csv.DictReader(f, delimiter=self.delimiter)

                for row in reader:
                    value = row[column_name_in_file]
                    cleaned_value = clean_value(value, value_type=data_type)

                    if isinstance(cleaned_value, list):
                        # If the value is a list, clean its items with the specified type in "type_if_list"
                        type_if_list = column_info.get("type_if_list", str)  # Use "type_if_list" if provided
                        for item in cleaned_value:
                            cleaned_item = clean_value(item, value_type=type_if_list)
                            if cleaned_item is not None:
                                unique_values.add(cleaned_item)
                    elif cleaned_value is not None:
                        unique_values.add(cleaned_value)

        except FileNotFoundError as e:
            print(f"Error: File not found: {e}")
            raise
        except csv.Error as e:
            print(f"Error reading the file: {e}")
            raise

        return [{db_column_name: value} for value in unique_values]

#### **`DataProcessor` Class**

In [35]:
class DataProcessor:
    def __init__(self, sqlite_connection, mysql_connection):
        """
        :param sqlite_connection: Connection object for SQLite
        :param mysql_connection: Connection object for MySQL
        """
        self.db = sqlite_connection  # SQLite connection
        self.mysql_db = mysql_connection  # MySQL connection

    def process_flat_tables(self, flat_tables_info):
        """Process each table and insert data as per the columns information."""

        for table_info in flat_tables_info:
            table_name = table_info["table_name"]  
            tsv_file = table_info["path_file"]
            delimiter = table_info["delimiter"]
            columns = table_info["columns"]
            unique_values = table_info["unique_values"] 
            
            print(f"Processing table: {table_name}")

            if unique_values:
                # If unique_values is True, insert only unique values
                self.__insert_unique_values(table_name, tsv_file, delimiter, columns)
            else:
                # If unique_values is False, process all values
                self.__process_and_insert_data(table_name, tsv_file, delimiter, columns)

    def process_regular_tables(self, regular_tables_info):
        for table_info in regular_tables_info:
            table_name = table_info["table_name"]  
            file = table_info["path_file"]
            delimiter = table_info["delimiter"]
            referenced_columns = table_info.get("referenced_columns", [])
            regular_columns = table_info["columns"]
            referenced_columns_with_values = []

            print(f"Processing table: {table_name}") 
            
            for column in referenced_columns:
                table_to_use = column['table_to_use']
                ref = column['columns_of_reference']
                column_values = {value : id 
                                 for value, id in self.db.fetch_records(
                                     table_to_use, [ref['column_in_table'],
                                                    ref['column_id_in_table']])
                                }
                ref["column_values"] = column_values
                ref["column_in_final_table"] = column['column_in_table']
                referenced_columns_with_values.append(ref)
            self.__process_and_insert_regular_tables(table_name, file, delimiter, regular_columns, referenced_columns_with_values)


    def process_join_tables(self, join_tables_info):
        for table_info in join_tables_info:
            table_name = table_info["table_name"]  
            file = table_info["path_file"]
            delimiter = table_info["delimiter"]
            referenced_columns = table_info.get("referenced_columns", [])
            regular_columns = table_info["columns"]
            referenced_columns_with_values = []
            print(f"Processing table: {table_name}") 
            
            for column in referenced_columns:
                table_to_use = column['table_to_use']
                default_value = column.get("default_value", None)
                ref = column['columns_of_reference']
                if default_value:
                    default_value = clean_value(default_value)
                    record = self.db.fetch_one_record(table_to_use,
                                                         [ref['column_in_table'],ref['column_id_in_table']],
                                                         {ref['column_in_table']:default_value})
                    if record:
                        value, id = record
                        column_values = {value : id}
                        ref["default_value"] = default_value
                    else:
                        print(f"The default_value {default_value} for the column {column} don't exists in the DB ")
                        raise sqlite3.Error
                        
                else:
                    column_values = {value : id 
                                     for value, id in self.db.fetch_records(
                                         table_to_use, [ref['column_in_table'],
                                                        ref['column_id_in_table']])
                                }
                ref["column_values"] = column_values
                ref["column_in_final_table"] = column['column_in_table']
                referenced_columns_with_values.append(ref)
            self.__process_and_insert_join_tables(table_name, file, delimiter, regular_columns, referenced_columns_with_values)

    def process_update_tables(self, tables_to_update_info):
        for table_info in tables_to_update_info:
            table_name = table_info["table_name"]  
            file = table_info["path_file"]
            delimiter = table_info["delimiter"]
            condition_columns = table_info["columns_for_condition"]
            regular_columns = table_info.get("columns",[])
            referenced_columns = table_info.get("referenced_columns",[])
            referenced_columns_with_values = []
            
            if len(condition_columns) > 0 and (len(regular_columns) + len(referenced_columns)) > 0:
                print(f"Processing table: {table_name}") 
                
                for column in referenced_columns:
                    table_to_use = column['table_to_use']
                    ref = column['columns_of_reference']
                    column_values = {value : id 
                                     for value, id in self.db.fetch_records(
                                         table_to_use, [ref['column_in_table'],
                                                        ref['column_id_in_table']])
                                    }
                    ref["column_values"] = column_values
                    ref["column_in_final_table"] = column['column_in_table']
                    referenced_columns_with_values.append(ref)
                
                self.__process_to_update_tables(table_name, file, delimiter, condition_columns, regular_columns, referenced_columns_with_values)
            else:
                print(f"You need 'columns for condition' for update the table <<{table_name}>>")


    def __process_to_update_tables(self, table_name, file, delimiter, condition_columns, regular_columns_to_update, referenced_columns_with_values):
        """Processes the file and update the data into the SQLite database."""
        print(f"Processing and update data into SQLite table '{table_name}' from {file}")
        
        file_processor = FileProcessor(file, delimiter)
        columns_to_update = []
        regular_columns = []
        regular_columns.extend(condition_columns)
        referenced_columns = []
        condition_columns_names = [column["column_in_table"] for column in condition_columns]
        if len(regular_columns_to_update) > 0:
            columns_to_update.extend([column["column_in_table"] for column in regular_columns_to_update])
            regular_columns.extend(regular_columns_to_update)
        if len(referenced_columns_with_values) > 0 : 
            columns_to_update.extend([column["column_in_final_table"] for column in referenced_columns_with_values])
            referenced_columns = referenced_columns_with_values
        
        for chunk in file_processor.read_in_chunks(regular_columns, referenced_columns):
            self.db.update_records(chunk, table_name, condition_columns_names, columns_to_update)
    
    def __process_and_insert_join_tables(self, table_name, file, delimiter, regular_columns, referenced_columns):
        """Processes the file and inserts the data into the SQLite database."""
        print(f"Processing and inserting data into SQLite table '{table_name}' from {file}")
        
        file_processor = FileProcessor(file, delimiter)
        columns_in_table = []
        if len(regular_columns) > 0:
            columns_in_table.extend([column["column_in_table"] for column in regular_columns])
        if len(referenced_columns) > 0 : 
            columns_in_table.extend([column["column_in_final_table"] for column in referenced_columns])
        for chunk in file_processor.read_in_chunks(regular_columns, referenced_columns, rows_with_list=True):
            self.db.insert_records(chunk, columns_in_table, table_name)

    
    def __process_and_insert_regular_tables(self, table_name, file, delimiter, regular_columns, referenced_columns):
        """Processes the file and inserts the data into the SQLite database."""
        print(f"Processing and inserting data into SQLite table '{table_name}' from {file}")
        file_processor = FileProcessor(file, delimiter)
        columns_in_table = [column["column_in_table"] for column in regular_columns]
        
        if len(referenced_columns) > 0 : 
            columns_in_table.extend([column["column_in_final_table"] for column in referenced_columns])
    
        for chunk in file_processor.read_in_chunks(regular_columns, referenced_columns):
            self.db.insert_records(chunk, columns_in_table, table_name)
    
    def __process_and_insert_data(self, table_name, tsv_file, delimiter, columns):
        """Processes the file and inserts the data into the SQLite database."""
        print(f"Processing and inserting data into SQLite table '{table_name}' from {tsv_file}")
        
        file_processor = FileProcessor(tsv_file, delimiter)
        columns_db = [column["column_in_table"] for column in columns]
        for chunk in file_processor.read_in_chunks(columns):      
            self.db.insert_records(chunk, columns_db, table_name)

    def __insert_unique_values(self, table_name, tsv_file, delimiter, columns):
        """Extracts and inserts unique values from a column into SQLite."""
        print(f"Inserting unique values into SQLite table '{table_name}' from {tsv_file}")

        # Assuming `columns` contains only one dictionary with the structure for unique values
        column_info = columns  # Here we take the first item, which is the only one for unique values
        file_processor = FileProcessor(tsv_file, delimiter)
        unique_values = file_processor.extract_unique_column_values(column_info)

        if unique_values:
            self.db.insert_records(unique_values, [column_info["column_in_table"]], table_name)

    def transfer_data_to_mysql(self, table_name):
        """Transfers data from SQLite to MySQL."""
        try:
            cursor = self.db.connection.cursor()
            query = f"SELECT * FROM {table_name}"
            cursor.execute(query)
            rows = cursor.fetchall()

            # Insert the data into MySQL
            # TO DO
            print(f"Data transferred from SQLite to MySQL for table '{table_name}'.")

        except sqlite3.Error as e:
            print(f"Error reading from SQLite: {e}")
        except Exception as e:
            print(f"Error during data transfer: {e}")

    def close(self):
        """Closes the connections to both databases."""
        if self.db:
            self.db.close()
            print("SQLite database connection closed.")
        if self.mysql_db:
            self.mysql_db.close()
            print("MySQL database connection closed.")

#### **Structure for flat tables `flat_tables_info`**

The order is very important for insertion in database because there are constraints in the tables.

```json

[
    {
        "table_name": <table_name_in_db>,  
        "path_file": <path_file>,
        "delimiter": <delimiter>,  # Delimiter of the file
        "columns": [
            {
                "column_in_file": <field_name_in_file>,
                "column_in_table": <field_name_in_db>,
                "column_type": <primitive_python_type_or_list>,
                "type_if_list": <primitive_python_type>
            },
            {
                "column_in_file": "name",
                "column_in_table": "name",
                "column_type": str
            },
            {
                "column_in_file": "age",
                "column_in_table": "age",
                "column_type": int
            }
        ],
        "unique_values": False 
    },
    {
        "table_name": "Subjects",
        "path_file": "../data/subjects.csv",
        "delimiter": ",",
        "columns":{
                "column_in_file": "subjects",
                "column_in_table": "suject",
                "column_type": list,
                "type_if_list": str
            }
        ,
        "unique_values": True  // If only one column and the values in the column are unique
    }
]

```

#### **`flat_tables_info`**

In [33]:
# Define the list of dictionaries
flat_tables_info = [
    {
        "path_file": '../Data_Files/data/title.basics.tsv',
        "table_name": 'Genre',
        "delimiter": "\t",
        "columns":{
            "column_in_file": "genres",
            "column_in_table": "genre_name",
            "column_type": list,
            "type_if_list": str
        },
        "unique_values": True
    },
    {
        "path_file": '../Data_Files/data/titles_types.csv',
        "table_name": 'Type',
        "delimiter": ",",
        "columns":[
            {
            "column_in_file": "id",
            "column_in_table": "type_id",
            "column_type": int
            },
            {
            "column_in_file": "type",
            "column_in_table": "type_name",
            "column_type": str,
            }
            
        ],
        "unique_values": False
    },
    {
        "path_file": '../Data_Files/data/title.basics.tsv',
        "table_name": 'Format',
        "delimiter": "\t",
        "columns":{
            "column_in_file": "titleType",
            "column_in_table": "format_name",
            "column_type": str
            },
        "unique_values": True
    },
    {
        "path_file": '../Data_Files/data/regions-codes.csv',
        "table_name": 'Region',
        "delimiter": ",",
        "columns":[
            {
            "column_in_file": "code",
            "column_in_table": "region_code",
            "column_type": str
            },
            {
            "column_in_file": "country",
            "column_in_table": "region_name",
            "column_type": str,
            }
            
        ],
        "unique_values": False
    },
    {
        "path_file": '../Data_Files/data/language-codes.csv',
        "table_name": 'Language',
        "delimiter": ",",
        "columns":[
            {
            "column_in_file": "code",
            "column_in_table": "language_code",
            "column_type": str
            },
            {
            "column_in_file": "name",
            "column_in_table": "language_name",
            "column_type": str,
            }
        ],
        "unique_values": False
    },
    {
        "path_file": '../Data_Files/data/name.basics.tsv',
        "table_name": 'Profession',
        "delimiter": "\t",
        "columns":{
            "column_in_file": "primaryProfession",
            "column_in_table": "profession_name",
            "column_type": list,
            "type_if_list": str
        },
        "unique_values": True
    },
    {
        "path_file": '../Data_Files/data/name.basics.tsv',
        "table_name": 'Person',
        "delimiter": "\t",
        "columns":[
            {
            "column_in_file": "nconst",
            "column_in_table": "person_id",
            "column_type": str
            },
            {
            "column_in_file": "primaryName",
            "column_in_table": "primary_name",
            "column_type": str,
            },
            {
            "column_in_file": "birthYear",
            "column_in_table": "birth_year",
            "column_type": int,
            },
            {
            "column_in_file": "deathYear",
            "column_in_table": "death_year",
            "column_type": int,
            }
        ],
        "unique_values": False
    },
    
]

#### **`regular_tables_info`**

In [234]:
# Define the list of dictionaries
regular_tables_info = [
    {
        "path_file": '../Data_Files/data/title.basics.tsv',
        "table_name": 'Title',
        "delimiter": "\t",
        "columns":[
            {
            "column_in_file": "tconst",
            "column_in_table": "title_id",
            "column_type": str
            },
            {
            "column_in_file": "originalTitle",
            "column_in_table": "original_title",
            "column_type": str,
            },
            {
            "column_in_file": "primaryTitle",
            "column_in_table": "primary_title",
            "column_type": str,
            },
            {
            "column_in_file": "isAdult",
            "column_in_table": "is_adult",
            "column_type": bool,
            },
            {
            "column_in_file": "startYear",
            "column_in_table": "start_year",
            "column_type": int,
            },
            {
            "column_in_file": "endYear",
            "column_in_table": "end_year",
            "column_type": int,
            },
            {
            "column_in_file": "runtimeMinutes",
            "column_in_table": "runtime",
            "column_type": int,
            },
        ],
        "referenced_columns":[
            {   "column_in_table": "format_id",
                "table_to_use": "Format",
                "columns_of_reference":{
                    "column_id_in_table" : "format_id",
                    "column_in_table" : "format_name",
                    "column_in_file" : "titleType",
                    "column_type" : str
                }                 
            }
        ]
    },
    {
        "path_file": '../Data_Files/data/title.ratings.tsv',
        "table_name": 'Rating',
        "delimiter": "\t",
        "columns":[
            {
            "column_in_file": "tconst",
            "column_in_table": "title_id",
            "column_type": str
            },
            {
            "column_in_file": "averageRating",
            "column_in_table": "average_rating",
            "column_type": str,
            },
            {
            "column_in_file": "numVotes",
            "column_in_table": "num_votes",
            "column_type": int,
            }   
        ]
    },
    {
        "path_file": '../Data_Files/data/title.episode.tsv',
        "table_name": 'Episode',
        "delimiter": "\t",
        "columns":[
            {
            "column_in_file": "tconst",
            "column_in_table": "episode_id",
            "column_type": str
            },
            {
            "column_in_file": "parentTconst",
            "column_in_table": "title_id",
            "column_type": str,
            },
            {
            "column_in_file": "seasonNumber",
            "column_in_table": "season_number",
            "column_type": int,
            },
            {
            "column_in_file": "episodeNumber",
            "column_in_table": "episode_number",
            "column_type": int,
            }
        ]
    },
    {
        "path_file": '../Data_Files/data/title.akas.tsv',
        "table_name": 'Regional_Title',
        "delimiter": "\t",
        "columns":[
            {
            "column_in_file": "titleId",
            "column_in_table": "title_id",
            "column_type": str
            },
            {
            "column_in_file": "title",
            "column_in_table": "regional_title",
            "column_type": str,
            },
            {
            "column_in_file": "isOriginalTitle",
            "column_in_table": "is_original_title",
            "column_type": bool,
            }
        ],
        "referenced_columns":[
            {   
                "column_in_table": "region_id",
                "table_to_use": "Region",
                "columns_of_reference":{
                    "column_id_in_table" : "region_id",
                    "column_in_table" : "region_code",
                    "column_in_file" : "region",
                    "column_type" : str
                }                 
            },
            {   
                "column_in_table": "language_id",
                "table_to_use": "Language",
                "columns_of_reference":{
                    "column_id_in_table" : "language_id",
                    "column_in_table" : "language_code",
                    "column_in_file" : "language",
                    "column_type" : str
                }                 
            }
        ]
    },
    {
        "path_file": '../Data_Files/data/production_companies.csv',
        "table_name": 'Production_company',
        "delimiter": "|",
        "columns":[
            {
            "column_in_file": "company_name",
            "column_in_table": "production_company_name",
            "column_type": str
            }
        ],
        "referenced_columns":[
            {   
                "column_in_table": "region_id",
                "table_to_use": "Region",
                "columns_of_reference":{
                    "column_id_in_table" : "region_id",
                    "column_in_table" : "region_code",
                    "column_in_file" : "region_code",
                    "column_type" : str
                }                 
            }
        ]
    }
    
]

#### **`join_tables_info`**

In [100]:
# Define the list of dictionaries
join_tables_info = [
    {
        "table_name": 'Genres_of_title',
        "path_file": '../Data_Files/data/title.basics.tsv',
        "delimiter": "\t",
        "columns":[
            {
                "column_in_file": "tconst",
                "column_in_table": "title_id",
                "column_type": str
            }
        ],
        "referenced_columns": [
            {   "column_in_table": "genre_id",
                "table_to_use": "Genre",
                "columns_of_reference":{
                    "column_id_in_table" : "genre_id",
                    "column_in_table" : "genre_name",
                    "column_in_file" : "genres",
                    "column_type" : list,
                    "type_if_list": str
                }                 
            }
        ]
    },
    {
        "table_name": 'Spoken_languages_in_title',
        "path_file": '../Data_Files/data/tmdb_full_separator.csv',
        "delimiter": "|",
        "columns":[
            {
                "column_in_file": "imdb_id",
                "column_in_table": "title_id",
                "column_type": str
            }
        ],
        "referenced_columns": [
            {   "column_in_table": "language_id",
                "table_to_use": "Language",
                "columns_of_reference":{
                    "column_id_in_table" : "language_id",
                    "column_in_table" : "language_code",
                    "column_in_file" : "spoken_languages",
                    "column_type" : list,
                    "type_if_list": str
                }                 
            }
        ]
    },
    {
        "table_name": 'Top_3_Professions',
        "path_file": '../Data_Files/data/name.basics.tsv',
        "delimiter": "\t",
        "columns":[
            {
                "column_in_file": "nconst",
                "column_in_table": "person_id",
                "column_type": str
            }
        ],
        "referenced_columns": [
            {   "column_in_table": "profession_id",
                "table_to_use": "Profession",
                "columns_of_reference":{
                    "column_id_in_table" : "profession_id",
                    "column_in_table" : "profession_name",
                    "column_in_file" : "primaryProfession",
                    "column_type" : list,
                    "type_if_list": str
                }                 
            }
        ]
    },
    {
        "table_name": 'Title_production_region',
        "path_file": '../Data_Files/data/tmdb_full_separator.csv',
        "delimiter": "|",
        "columns":[
            {
                "column_in_file": "imdb_id",
                "column_in_table": "title_id",
                "column_type": str
            }
        ],
        "referenced_columns": [
            {   "column_in_table": "region_id",
                "table_to_use": "Region",
                "columns_of_reference":{
                    "column_id_in_table" : "region_id",
                    "column_in_table" : "region_code",
                    "column_in_file" : "production_countries",
                    "column_type" : list,
                    "type_if_list": str
                }                 
            }
        ]
    },
    {
        "table_name": 'Title_by_production_company',
        "path_file": '../Data_Files/data/tmdb_full_separator.csv',
        "delimiter": "|",
        "columns":[
            {
                "column_in_file": "imdb_id",
                "column_in_table": "title_id",
                "column_type": str
            }
        ],
        "referenced_columns": [
            {   "column_in_table": "production_company_id",
                "table_to_use": "Production_company",
                "columns_of_reference":{
                    "column_id_in_table" : "production_company_id",
                    "column_in_table" : "production_company_name",
                    "column_in_file" : "production_companies_name",
                    "column_type" : list,
                    "type_if_list": str
                }                 
            }
        ]
    },
    {
        "table_name": 'Crew',
        "path_file": '../Data_Files/data/title.crew.tsv',
        "delimiter": "\t",
        "columns":[
            {
                "column_in_file": "tconst",
                "column_in_table": "title_id",
                "column_type": str
            },
            {
                "column_in_file": "directors",
                "column_in_table": "person_id",
                "column_type": list,
                "type_if_list": str
            }
        ],
        "referenced_columns": [
            {   
                "column_in_table": "profession_id",
                "table_to_use": "Profession",
                "default_value": "director",
                "columns_of_reference":{
                    "column_id_in_table" : "profession_id",
                    "column_in_table" : "profession_name",
                }                 
            }
        ]
    },
    {
        "table_name": 'Crew',
        "path_file": '../Data_Files/data/title.crew.tsv',
        "delimiter": "\t",
        "columns":[
            {
                "column_in_file": "tconst",
                "column_in_table": "title_id",
                "column_type": str
            },
            {
                "column_in_file": "writers",
                "column_in_table": "person_id",
                "column_type": list,
                "type_if_list": str
            }
        ],
        "referenced_columns": [
            {   
                "column_in_table": "profession_id",
                "table_to_use": "Profession",
                "default_value": "writer",
                "columns_of_reference":{
                    "column_id_in_table" : "profession_id",
                    "column_in_table" : "profession_name",
                }                 
            }
        ]
    },
    {
        "table_name": 'Crew',
        "path_file": '../Data_Files/data/title.principals.tsv',
        "delimiter": "\t",
        "columns":[
            {
                "column_in_file": "tconst",
                "column_in_table": "title_id",
                "column_type": str
            },
            {
                "column_in_file": "nconst",
                "column_in_table": "person_id",
                "column_type": str,
            }
        ],
        "referenced_columns": [
            {   
                "column_in_table": "profession_id",
                "table_to_use": "Profession",
                "columns_of_reference":{
                    "column_id_in_table" : "profession_id",
                    "column_in_table" : "profession_name",
                    "column_in_file" : "category",
                    "column_type" : str,
                }                 
            }
        ]
    }
]

#### **`table_to_update_info`**

In [53]:
# Define the list of dictionaries
tables_to_update_info = [
    {
        "table_name": 'Title',
        "path_file": '../Data_Files/data/tmdb_full_separator.csv',
        "delimiter": "|",
        "columns_for_condition":[
            {
                "column_in_file": "imdb_id",
                "column_in_table": "title_id",
                "column_type": str
            }
        ],
        "columns":[
            {
                "column_in_file": "budget",
                "column_in_table": "budget",
                "column_type": int
            },
            {
                "column_in_file": "revenue",
                "column_in_table": "revenue",
                "column_type": int
            },
            {
                "column_in_file": "overview",
                "column_in_table": "overview",
                "column_type": str
            }
        ]
    } 
]

#### **`main` function**

In [1]:
# Main function to orchestrate the entire process
def main():
    # SQLite connection
    sqlite_connection = SQLiteDatabase(sqlite3_db)

    # MySQL connection
    mysql_connection = MySQLDatabase(
        host="localhost",
        user="root",
        password="password",
        database="mysql_database"
    )

    # Populate the SQLite database and transfer data to MySQL
    data_processor = DataProcessor(sqlite_connection, mysql_connection)

    # Process and insert data
    data_processor.process_flat_tables(flat_tables_info)
    data_processor.process_regular_tables(regular_tables_info)
    data_processor.process_join_tables(join_tables_info)
    data_processor.process_update_tables(tables_to_update_info)

    # Close connections
    data_processor.close()



In [ ]:
main()

# KPIs

## WEIGHTED_RATING VIEW

```sql
CREATE VIEW Weighted_rating AS
WITH min_votes AS (
	WITH Rating_Full AS (
		Select num_votes, ntile(100) over(order by num_votes ) as percentile 
		from Rating WHERE num_votes IS NOT NULL)
	SELECT MAX(num_votes) AS min_votes FROM Rating_Full WHERE percentile = 95
),
mean_rating AS (
	SELECT AVG(average_rating) AS mean_rating FROM Rating
)
SELECT r.rating_id, r.title_id, r.num_votes, r.average_rating, ((r.num_votes*r.average_rating)/ (r.num_votes + m.min_votes) + (m.min_votes*c.mean_rating)/ (r.num_votes + m.min_votes))  AS weighted_rating 
FROM Rating AS r, min_votes as m, mean_rating as c ORDER BY weighted_rating DESC;
```

## KPIs Finances

### Rentabilidad Promedio de las Producciones (Revenue / Budget)

- __SQL__

```sql
SELECT t.primary_title, t.revenue, t.budget, 
      (t.revenue / t.budget) AS profitability
FROM Title t
WHERE t.budget > 0;
```

- __KPI__ : Bar

### Revenue moyen par genre

- __SQL__

```sql
SELECT g.genre_name, AVG(t.revenue) AS average_revenue
FROM Title t
JOIN Genres_of_title gt ON t.title_id = gt.title_id
JOIN Genre g ON gt.genre_id = g.genre_id
GROUP BY g.genre_name;
``

- __KPI__ : Stack Bar

### Revenue total par year
   

- __SQL__

```sql
SELECT t.start_year, SUM(t.revenue) AS total_revenue
FROM Title t
GROUP BY t.start_year
ORDER BY t.start_year;
```

- __KPI__ : Line

### Titres avec le plus haut revenue

- __SQL__

```sql
SELECT t.primary_title, t.revenue
FROM Title t
ORDER BY t.revenue DESC
LIMIT 10;
```

- __KPI__ : Horizontal Bar

### Pourcentage de Titles par range 0 < 1 < 10 < 50 < 100 des revenues

- __SQL__
  
```sql
SELECT (COUNT(*) * 100.0) / (SELECT COUNT(*) FROM Title) AS percentage_above_100M
FROM Title
WHERE revenue > 100000000;
``

- __KPI__ : Pie

### Rating moyen par Revenue (Rating vs. Revenue)

- __SQL__
  
```sql
SELECT t.primary_title, r.average_rating, t.revenue
FROM Title t
JOIN Rating r ON t.title_id = r.title_id;
``

- __KPI__ : Scatter

### Total de Títulos por Rango de Ingresos

- __SQL__

```sql
SELECT 
   CASE 
       WHEN t.revenue < 1000000 THEN 'Less than 1M'
       WHEN t.revenue BETWEEN 1000000 AND 10000000 THEN '1M - 10M'
       WHEN t.revenue BETWEEN 10000000 AND 100000000 THEN '10M - 100M'
       ELSE 'More than 100M'
   END AS revenue_range,
   COUNT(*) AS total_titles
FROM Title t
GROUP BY revenue_range;
```

- __KPI__ : Stack Bar

### Títulos con el Mejor Margen de Rentabilidad (Revenue - Budget)

- __SQL__

```sql
SELECT t.primary_title, (t.revenue - t.budget) AS profit_margin
FROM Title t
WHERE t.budget > 0
ORDER BY profit_margin DESC
LIMIT 10;
```

- __KPI__ : Horizontal bar

### Ingresos por Compañía Productora

- __SQL__

```sql
SELECT pc.production_company_name, SUM(t.revenue) AS total_revenue
FROM Production_company pc
JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
JOIN Title t ON tpc.title_id = t.title_id
GROUP BY pc.production_company_name;
```

- __KPI__

### Ingresos Promedio por Género y Año

- __SQL__

```sql
SELECT g.genre_name, t.start_year, AVG(t.revenue) AS avg_revenue
FROM Title t
JOIN Genres_of_title gt ON t.title_id = gt.title_id
JOIN Genre g ON gt.genre_id = g.genre_id
GROUP BY g.genre_name, t.start_year
ORDER BY t.start_year;
```

- __KPI__


## KPIs de Recursos Humanos

### Número de Profesionales Involucrados en la Industria

- __SQL__

```sql
SELECT COUNT(DISTINCT person_Id) AS total_professionals
FROM Crew;
```

- __KPI__

### Profesiones Más Comunes en las Producciones

- __SQL__

```sql
SELECT pr.profession_name, COUNT(*) AS count
FROM Crew c
JOIN Profession pr ON c.profession_id = pr.profession_id
GROUP BY pr.profession_name
ORDER BY count DESC
LIMIT 5;
```

- __KPI__

### Promedio de Participación por Persona

- __SQL__

```sql
SELECT p.primary_name, COUNT(DISTINCT c.title_id) AS titles_participated
FROM Person p
JOIN Crew c ON p.person_Id = c.person_Id
GROUP BY p.person_Id;
```

- __KPI__

### Top 3 Profesionales Más Involucrados (Número de Títulos)

- __SQL__

```sql
SELECT p.primary_name, COUNT(DISTINCT c.title_id) AS titles_count
FROM Person p
JOIN Crew c ON p.person_Id = c.person_Id
GROUP BY p.person_Id
ORDER BY titles_count DESC
LIMIT 3;
```

- __KPI__

### Profesionales más Exitosos por Rating Promedio

- __SQL__

```sql
SELECT p.primary_name, AVG(r.average_rating) AS avg_rating
FROM Person p
JOIN Crew c ON p.person_Id = c.person_Id
JOIN Title t ON c.title_id = t.title_id
JOIN Rating r ON t.title_id = r.title_id
GROUP BY p.primary_name
ORDER BY avg_rating DESC
LIMIT 5;
```

- __KPI__

### Número de Películas por Profesional y Profesión

- __SQL__

```sql
SELECT p.primary_name, pr.profession_name, COUNT(DISTINCT c.title_id) AS total_titles
FROM Person p
JOIN Crew c ON p.person_Id = c.person_Id
JOIN Profession pr ON c.profession_id = pr.profession_id
GROUP BY p.primary_name, pr.profession_name;
```

- __KPI__

### Promedio de Participación de Profesionales por Género

- __SQL__

```sql
SELECT pr.profession_name, g.genre_name, COUNT(DISTINCT c.title_id) / COUNT(DISTINCT p.person_Id) AS avg_participation
FROM Crew c
JOIN Profession pr ON c.profession_id = pr.profession_id
JOIN Person p ON c.person_Id = p.person_Id
JOIN Genres_of_title gt ON c.title_id = gt.title_id
JOIN Genre g ON gt.genre_id = g.genre_id
GROUP BY pr.profession_name, g.genre_name;
```

- __KPI__

### Número de Profesionales por Región

- __SQL__

```sql
SELECT r.region_name, COUNT(DISTINCT c.person_Id) AS total_professionals
FROM Region r
JOIN Production_company pc ON r.region_id = pc.region_id
JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
JOIN Crew c ON tpc.title_id = c.title_id
GROUP BY r.region_name;
```

- __KPI__

### Profesionales por Título (Promedio de Profesionales)

- __SQL__

```sql
SELECT t.primary_title, COUNT(DISTINCT c.person_Id) AS professionals_count
FROM Title t
JOIN Crew c ON t.title_id = c.title_id
GROUP BY t.title_id;
```

- __KPI__

### Número de Profesionales por Año de Producción

- __SQL__

```sql
SELECT t.start_year, COUNT(DISTINCT c.person_Id) AS total_professionals
FROM Title t
JOIN Crew c ON t.title_id = c.title_id
GROUP BY t.start_year;
```

- __KPI__

## KPIs de Países y Lenguajes

### Número de Títulos por Región

- __SQL__

```sql
SELECT r.region_name, COUNT(DISTINCT t.title_id) AS titles_count
FROM Region r
JOIN Title_production_region tpr ON r.region_id = tpr.region_id
JOIN Title t ON tpr.title_id = t.title_id
GROUP BY r.region_name;
```

- __KPI__

### Distribución de Idiomas por Título

- __SQL__

```sql
SELECT t.primary_title, COUNT(DISTINCT sl.language_id) AS languages_count
FROM Title t
JOIN Spoken_languages_in_title sl ON t.title_id = sl.title_id
GROUP BY t.title_id;
```

- __KPI__

### Porcentaje de Títulos con Múltiples Idiomas

- __SQL__

```sql
SELECT (COUNT(DISTINCT t.title_id) * 100.0) / (SELECT COUNT(*) FROM Title) AS percentage_multi_language
FROM Title t
JOIN Spoken_languages_in_title sl ON t.title_id = sl.title_id
GROUP BY t.title_id
HAVING COUNT(DISTINCT sl.language_id) > 1;
```

### 4. Promedio de Títulos por Región
   SELECT AVG(title_count) AS avg_titles_per_region
   FROM (SELECT r.region_name, COUNT(DISTINCT t.title_id) AS title_count
         FROM Region r
         JOIN Title_production_region tpr ON r.region_id = tpr.region_id
         JOIN Title t ON tpr.title_id = t.title_id
         GROUP BY r.region_name) AS region_titles;

### 5. Número de Idiomas por Región
   SELECT r.region_name, COUNT(DISTINCT sl.language_id) AS languages_count
   FROM Region r
   JOIN Title_production_region tpr ON r.region_id = tpr.region_id
   JOIN Title t ON tpr.title_id = t.title_id
   JOIN Spoken_languages_in_title sl ON t.title_id = sl.title_id
   GROUP BY r.region_name;

### 6. Top 5 Idiomas Más Comunes
   SELECT l.language_name, COUNT(*) AS language_count
   FROM Spoken_languages_in_title sl
   JOIN Language l ON sl.language_id = l.language_id
   GROUP BY l.language_name
   ORDER BY language_count DESC
   LIMIT 5;

### 7. Títulos Doblados vs. Subtitulados
   SELECT 
       CASE
           WHEN rt.is_original_title = 1 THEN 'Original'
           ELSE 'Doblado/Subtitulado'
       END AS title_type,
       COUNT(DISTINCT t.title_id) AS titles_count
   FROM Regional_Title rt
   JOIN Title t ON rt.title_id = t.title_id
   GROUP BY title_type;

### 8. Títulos por Región y Idioma
   SELECT r.region_name, l.language_name, COUNT(DISTINCT t.title_id) AS titles_count
   FROM Region r
   JOIN Title_production_region tpr ON r.region_id = tpr.region_id
   JOIN Title t ON tpr.title_id = t.title_id
   JOIN Spoken_languages_in_title sl ON t.title_id = sl.title_id
   JOIN Language l ON sl.language_id = l.language_id
   GROUP BY r.region_name, l.language_name;

### 9. Títulos en Idiomas No Nativos (Por Región)
   SELECT r.region_name, COUNT(DISTINCT t.title_id) AS non_native_titles
   FROM Region r
   JOIN Title_production_region tpr ON r.region_id = tpr.region_id
   JOIN Title t ON tpr.title_id = t.title_id
   JOIN Regional_Title rt ON t.title_id = rt.title_id
   WHERE rt.is_original_title = 0
   GROUP BY r.region_name;

### 10. Número de Idiomas en los Títulos por Año
    SELECT t.start_year, COUNT(DISTINCT sl.language_id) AS languages_count
    FROM Title t
    JOIN Spoken_languages_in_title sl ON t.title_id = sl.title_id
    GROUP BY t.start_year;

## KPIs de Compañías Productoras

### 1. Total de Ingresos por Compañía Productora
   SELECT pc.production_company_name, SUM(t.revenue) AS total_revenue
   FROM Production_company pc
   JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
   JOIN Title t ON tpc.title_id = t.title_id
   GROUP BY pc.production_company_name;

### 2. Número de Títulos por Compañía Productora
   SELECT pc.production_company_name, COUNT(*) AS total_titles
   FROM Production_company pc
   JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
   JOIN Title t ON tpc.title_id = t.title_id
   GROUP BY pc.production_company_name;

### 3. Top 10 Compañías Productoras con Mayor Ingreso
   SELECT pc.production_company_name, SUM(t.revenue) AS total_revenue
   FROM Production_company pc
   JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
   JOIN Title t ON tpc.title_id = t.title_id
   GROUP BY pc.production_company_name
   ORDER BY total_revenue DESC
   LIMIT 10;

### 4. Compañías Productoras con Más Títulos
   SELECT pc.production_company_name, COUNT(*) AS total_titles
   FROM Production_company pc
   JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
   JOIN Title t ON tpc.title_id = t.title_id
   GROUP BY pc.production_company_name
   ORDER BY total_titles DESC
   LIMIT 10;

### 5. Promedio de Ingresos por Título por Compañía Productora
   SELECT pc.production_company_name, AVG(t.revenue) AS avg_revenue_per_title
   FROM Production_company pc
   JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
   JOIN Title t ON tpc.title_id = t.title_id
   GROUP BY pc.production_company_name;

### 6. Total de Ingresos por Compañía Productora y Año
   SELECT pc.production_company_name, t.start_year, SUM(t.revenue) AS total_revenue
   FROM Production_company pc
   JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
   JOIN Title t ON tpc.title_id = t.title_id
   GROUP BY pc.production_company_name, t.start_year
   ORDER BY t.start_year;

### 7. Top 5 Compañías Productoras por Rating Promedio
   SELECT pc.production_company_name, AVG(r.average_rating) AS avg_rating
   FROM Production_company pc
   JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
   JOIN Title t ON tpc.title_id = t.title_id
   JOIN Rating r ON t.title_id = r.title_id
   GROUP BY pc.production_company_name
   ORDER BY avg_rating DESC
   LIMIT 5;

### 8. Ingresos por Compañía Productora en los Últimos 5 Años
   SELECT pc.production_company_name, SUM(t.revenue) AS total_revenue
   FROM Production_company pc
   JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
   JOIN Title t ON tpc.title_id = t.title_id
   WHERE t.start_year >= (YEAR(CURDATE()) - 5)
   GROUP BY pc.production_company_name;

### 9. Promedio de Títulos por Compañía Productora por Año
   SELECT pc.production_company_name, t.start_year, COUNT(*) / COUNT(DISTINCT t.start_year) AS avg_titles_per_year
   FROM Production_company pc
   JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
   JOIN Title t ON tpc.title_id = t.title_id
   GROUP BY pc.production_company_name, t.start_year;

### 10. Compañías Productoras y sus Títulos Más Rentables
    SELECT pc.production_company_name, t.primary_title, t.revenue
    FROM Production_company pc
    JOIN Title_by_production_company tpc ON pc.production_company_id = tpc.production_company_id
    JOIN Title t ON tpc.title_id = t.title_id
    ORDER BY t.revenue DESC
    LIMIT 10;

# __SYSTEM RECOMMENDATION ENGINE__

## ETL

### __Python imports__

In [ ]:
import pandas as pd

In [ ]:
tmdb = pd.read_csv(filepath_or_buffer="../Data_Files/data/TMDB_movie_dataset_v11.csv", sep=",")

In [ ]:
crew = pd.read_csv(filepath_or_buffer="../Data_Files/data/title.crew.tsv.gz",compression="gzip", sep="\t")

In [ ]:
people = pd.read_csv(filepath_or_buffer="../Data_Files/data/name.basics.tsv.gz",compression="gzip", sep="\t")

### __UTILS FUNCTIONS__ 

In [ ]:
def print_na_by_columns(dataframe: pd.DataFrame):
    flag = True
    for column in dataframe.columns:
        count = dataframe[column].isna().sum()
        if count > 0:
            flag = False
            print(f"The column {column} has {count} null cells ")
    if flag:
        print(f"There aren't null values")

In [ ]:
def print_basic_analysis(dataframe: pd.DataFrame, dataframe_name: str = 'DATAFRAME'):
    print(f"-------------------------------------------------------------------------------------")
    print(f"------------------------------{dataframe_name}---------------------------------------")
    print(dataframe.head(5), end="\n\n")
    print(dataframe.info(), end="\n\n")
    print_na_by_columns(dataframe)

In [ ]:
def remove_duplicates_by_column(dataframe: pd.DataFrame, columns: list) -> pd.DataFrame:
    for column in columns:
        dataframe = dataframe.loc[~dataframe.duplicated(subset=column)]
    return dataframe

### __INITIAL ANALYSIS__

In [ ]:
print_basic_analysis(tmdb, 'TMDB')

In [ ]:
print_basic_analysis(crew, 'CREW')

In [ ]:
print_basic_analysis(people, 'PEOPLE')

### __ONLY NECESSARY COLUMNS__

In [ ]:
tmdb_columns = ['id', 'title', 'vote_average', 'vote_count', 'release_date', 'poster_path',
       'imdb_id', 'original_title', 'overview',
       'popularity', 'tagline', 'genres','keywords']
tmdb = tmdb[tmdb_columns]

In [ ]:
crew_columns = ['tconst', 'directors']
crew = crew[crew_columns]

In [ ]:
people_columns = ['nconst', 'primaryName']
people = people[people_columns]

### __REMOVE ROWS WITH NULL VALUES__

In [ ]:
colums_not_na = ['id', 'title', 'vote_average', 'vote_count',
       'imdb_id', 'original_title', 'overview',
       'popularity', 'genres']
tmdb.dropna(subset=colums_not_na, inplace=True, axis=0)

print(tmdb.info(), end="\n\n")
print_na_by_columns(tmdb)

In [ ]:
people.dropna(inplace=True, axis=0)

print(people.info(), end="\n\n")
print_na_by_columns(people)

### __REMOVING DUPLICATES COLUMS__

In [ ]:
print(f"TMDB size before: {len(tmdb)}")
tmdb = remove_duplicates_by_column(tmdb, ['id', 'imdb_id'])
print(f"TMDB size after: {len(tmdb)}")

In [ ]:
print(f"PEOPLE size before: {len(people)}")
people = remove_duplicates_by_column(people, ['nconst'])
print(f"PEOPLE size after: {len(people)}")

In [ ]:
print(f"CREW size before: {len(crew)}")
crew = remove_duplicates_by_column(crew, ['tconst'])
print(f"CREW size after: {len(crew)}")

### __JOIN DATAFRAMES__

In [ ]:
tmdb = pd.merge(tmdb, crew, how='left', left_on='imdb_id', right_on='tconst').drop(columns=['imdb_id', 'tconst'])
tmdb.info()

In [ ]:
people.set_index('nconst', inplace=True)

In [ ]:
def get_primary_name(nconst:str):
    primary_name = people.loc[nconst, 'primaryName']
    return primary_name if primary_name != '' else None

def get_primary_names(nconsts: str) -> str:
    names_list = [get_primary_name(str(nconst).strip()) for nconst in nconsts.split(',') if nconst.strip() not in ['\\N', '']]
    return ','.join(names_list)

In [ ]:
tmdb.fillna({'directors':''}, inplace=True)

In [ ]:
tmdb['directors'] = tmdb['directors'].apply(get_primary_names)

In [ ]:
tmdb.fillna({'genres':''}, inplace=True)

### __SAVE DATAFRAME TO CSV FILE__

In [ ]:
tmdb.to_csv(path_or_buf="../Data_Files/data/TMDB_movies_full.csv", index=False)

## NLP

### __Python imports__

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack
import joblib

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gabriel-
[nltk_data]     santisteban/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### __Utils Functions__

In [3]:
def clean_str(text: str) -> str:
    text = text.strip().lower()
    text = re.sub(r'[^a-z\s]'," ", text)
    return text

In [4]:
def split_and_clean_str(x: str, delimiter=",", remove_spaces: bool=False):
    new_list = []
    if isinstance(x, str) and x != None and len(x.strip()) > 0 and x !='\\N' and x != np.nan:
        new_list = [clean_str(val) for val in x.split(delimiter) if val.strip() != ""]
    if isinstance(x, list):
        new_list = [clean_str(val) for val in x if val and val.strip() != ""]
    if remove_spaces:
        new_list = [val.replace(" ", "") for val in new_list]
    return new_list

In [5]:
def filter_keywords(keywords: list, interesting_keywords: list):
    words = []
    for word in keywords:
        if word in interesting_keywords:
            words.append(word)
    return words

In [6]:
tokenizer = RegexpTokenizer(r'\w+').tokenize
def split_with_tokenizer(text: str) -> list:
    if isinstance(text, str):
        return tokenizer(text)
    return []

In [7]:
def word_lists_to_str(word_lists: list, weight) -> str:
    return " ".join([word for word_list in word_lists  
              for word in word_list*weight if word not in stopwords.words('english')])

### __Loading Data__

In [8]:
tmdb_full = pd.read_csv(filepath_or_buffer="../data/TMDB_movies_full.csv", sep=",")

In [9]:
tmdb_full.head()

,id,title,vote_average,vote_count,original_title,overview,popularity,tagline,genres,keywords,directors
0,27205,Inception,8.364,34495,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","rescue, mission, dream, airplane, paris, franc...",Christopher Nolan
1,157336,Interstellar,8.417,32571,Interstellar,The adventures of a group of explorers who mak...,140.241,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","rescue, future, spacecraft, race against time,...",Christopher Nolan
2,155,The Dark Knight,8.512,30619,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","joker, sadism, chaos, secret identity, crime f...",Christopher Nolan
3,19995,Avatar,7.573,29815,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","future, society, culture clash, space travel, ...",James Cameron
4,24428,The Avengers,7.710,29166,The Avengers,When an unexpected enemy emerges and threatens...,98.082,Some assembly required.,"Science Fiction, Action, Adventure","new york city, superhero, shield, based on com...",Joss Whedon


In [10]:
tmdb_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404014 entries, 0 to 404013
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              404014 non-null  int64  
 1   title           404014 non-null  object 
 2   vote_average    404014 non-null  float64
 3   vote_count      404014 non-null  int64  
 4   original_title  404014 non-null  object 
 5   overview        404014 non-null  object 
 6   popularity      404014 non-null  float64
 7   tagline         94612 non-null   object 
 8   genres          404014 non-null  object 
 9   keywords        178158 non-null  object 
 10  directors       390423 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 33.9+ MB


### __Transforming columns__

#### __From str to list__

In [11]:
tmdb_full.fillna({'directors':'', 'genres':'', 'keywords':'', 'tagline': '', 'overview':''}, inplace=True)

In [12]:
tmdb_full['directors'] = tmdb_full['directors'].apply(lambda x: split_and_clean_str(x, remove_spaces=True))

In [13]:
tmdb_full['genres'] = tmdb_full['genres'].apply(lambda x: split_and_clean_str(x, remove_spaces=True))

In [14]:
tmdb_full['keywords'] = tmdb_full['keywords'].apply(split_and_clean_str)

In [15]:
tmdb_full['overview'] = tmdb_full['overview'].apply(split_with_tokenizer)

In [16]:
tmdb_full['tagline'] = tmdb_full['tagline'].apply(split_with_tokenizer)

#### __Removing `keywords` not interesting -> `keyword occurrences = 1`__

In [17]:
all_keywords = tmdb_full['keywords'].explode().to_list()
all_keywords = sorted([key for key, value in nltk.FreqDist(all_keywords).items() if value > 1])

In [18]:
tmdb_full['keywords'] = tmdb_full['keywords'].apply(lambda x: filter_keywords(x, all_keywords))

#### __Steaming on `keywords`, `description` and `tagline` and remove spaces between words__

In [19]:
stemmer = SnowballStemmer(language='english')

In [20]:
tmdb_full['keywords'] = tmdb_full['keywords'].apply(lambda x: [stemmer.stem(i).replace(" ", "") for i in x])

In [21]:
tmdb_full['tagline'] = tmdb_full['tagline'].apply(lambda x: [stemmer.stem(i) for i in x])

In [22]:
tmdb_full['overview'] = tmdb_full['overview'].apply(lambda x: [stemmer.stem(i) for i in x])

## ML

### __Vectorizing the text__

#### __Using `CountVector` on columns__

In [23]:
columns_weight = {'overview':1, 'directors':3, 'tagline':1, 'keywords':1, 'genres':2}

In [24]:
vectorizers = {col:(CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english'), weight) 
               for col,weight in columns_weight.items()}

In [25]:
print(vectorizers)

{'overview': (CountVectorizer(min_df=0.0, ngram_range=(1, 2), stop_words='english'), 1), 'directors': (CountVectorizer(min_df=0.0, ngram_range=(1, 2), stop_words='english'), 3), 'tagline': (CountVectorizer(min_df=0.0, ngram_range=(1, 2), stop_words='english'), 1), 'keywords': (CountVectorizer(min_df=0.0, ngram_range=(1, 2), stop_words='english'), 1), 'genres': (CountVectorizer(min_df=0.0, ngram_range=(1, 2), stop_words='english'), 2)}


In [26]:
all_matrix = []
for col, (vect, weight) in vectorizers.items():
    all_matrix.append(vect.fit_transform(tmdb_full[col].apply(lambda x: " ".join(x*weight))))

#### __Stacking the matrices__

In [27]:
all_matrices = hstack(all_matrix)

### __KNN : Creating the Model__

In [28]:
modelKNN = NearestNeighbors(n_neighbors=10, metric='cosine')
modelKNN.fit(all_matrices)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [29]:
distances, indices = modelKNN.kneighbors(all_matrices[0], n_neighbors=20)

In [30]:
tmdb_full.loc[indices[0]]

,id,title,vote_average,vote_count,original_title,overview,popularity,tagline,genres,keywords,directors
0,27205,Inception,8.364,34495,Inception,"[cobb, a, skill, thief, who, commit, corpor, e...",83.952,"[your, mind, is, the, scene, of, the, crime]","[action, sciencefiction, adventure]","[rescu, mission, dream, airplan, pari, franc, ...",[christophernolan]
333127,83533,Avatar 3,0.000,0,Avatar 3,"[the, third, entri, in, the, avatar, franchis]",14.635,[],"[action, sciencefiction, adventure]",[sequel],[jamescameron]
19208,10921,Babylon 5: The River of Souls,6.374,91,Babylon 5: The River of Souls,"[a, group, of, soul, hunter, come, to, babylon...",8.387,[],"[action, sciencefiction, adventure]","[alienlifeform, womandirector, spacecentr]",[janetgreek]
195890,750535,Monochrome: The Chromism,1.000,1,Monochrome: The Chromism,"[trade, and, sold, like, currenc, the, outcast...",0.600,[],"[action, sciencefiction, adventure]",[],[kodizene]
321463,278238,Z,0.000,0,Z,"[the, western, frontier, reemerg, and, as, in,...",2.459,"[when, vengeanc, becom, justic]","[adventure, action, sciencefiction]",[],[jonscuarn]
160014,651692,Virtual Death Match,6.000,2,Virtual Death Match,"[a, group, of, gamer, are, select, to, take, p...",3.470,[],"[horror, action, sciencefiction, adventure]",[],[louisawarren]
18345,10916,Babylon 5: A Call to Arms,6.816,98,Babylon 5: A Call to Arms,"[alli, of, the, shadow, seek, reveng, against,...",8.286,[],"[action, sciencefiction, adventure]","[spacemarin, nightmar, spacewar, alienlifeform...",[michaelvejar]
10364,240483,Robot Overlords,4.900,241,Robot Overlords,"[earth, has, been, conquer, by, robot, from, a...",12.406,"[mankind, may, fall, hero, will, rise]","[sciencefiction, adventure, action]","[invas, robot]",[jonwright]
401778,760328,Robopocalypse,0.000,0,Robopocalypse,"[in, a, futurist, world, a, human, resist, mov...",0.600,[],"[sciencefiction, adventure, action]",[],[michaelbay]
61976,74839,Star Odyssey,2.200,14,Sette uomini d'oro nello spazio,"[earth, is, attack, by, an, intergalact, villa...",2.542,"[the, earth, s, ultim, disast]","[sciencefiction, adventure, action]","[alien, robot, spaceopera, outerspac]",[alfonsobrescia]


#### __Testing the model__

In [38]:
test = tmdb_full.loc[[0, 7210, 354]]
test.head()

,id,title,vote_average,vote_count,original_title,overview,popularity,tagline,genres,keywords,directors
0,27205,Inception,8.364,34495,Inception,"[cobb, a, skill, thief, who, commit, corpor, e...",83.952,"[your, mind, is, the, scene, of, the, crime]","[action, sciencefiction, adventure]","[rescu, mission, dream, airplan, pari, franc, ...",[christophernolan]
7210,43629,Doodlebug,6.500,422,Doodlebug,"[in, his, squalid, apart, a, man, tri, to, squ...",5.715,[],[horror],"[blackandwhit, bug, shoe, shortfilm]",[christophernolan]
354,577922,Tenet,7.191,8744,Tenet,"[arm, with, onli, one, word, tenet, and, fight...",44.025,"[time, run, out]","[action, thriller, sciencefiction]","[assassin, espionag, spi, timetravel, mumbaibo...",[christophernolan]


In [39]:
test_matrix = []

for col,(vect, weight) in vectorizers.items():
    col_value = [word_lists_to_str(test[col].to_list(), weight)]
    vector = vect.transform(col_value)
    test_matrix.append(vector)

In [40]:
test_matrices = hstack(test_matrix)

In [41]:
print(test_matrices.shape)

(1, 5963281)


In [42]:
distances, indices = modelKNN.kneighbors(test_matrices, n_neighbors=1000)

In [44]:
tmdb_full.loc[indices[0]].head(10)

,id,title,vote_average,vote_count,original_title,overview,popularity,tagline,genres,keywords,directors
0,27205,Inception,8.364,34495,Inception,"[cobb, a, skill, thief, who, commit, corpor, e...",83.952,"[your, mind, is, the, scene, of, the, crime]","[action, sciencefiction, adventure]","[rescu, mission, dream, airplan, pari, franc, ...",[christophernolan]
354,577922,Tenet,7.191,8744,Tenet,"[arm, with, onli, one, word, tenet, and, fight...",44.025,"[time, run, out]","[action, thriller, sciencefiction]","[assassin, espionag, spi, timetravel, mumbaibo...",[christophernolan]
7210,43629,Doodlebug,6.500,422,Doodlebug,"[in, his, squalid, apart, a, man, tri, to, squ...",5.715,[],[horror],"[blackandwhit, bug, shoe, shortfilm]",[christophernolan]
88982,456684,Larceny,5.700,7,Larceny,"[a, man, break, into, a, flat, startl, the, oc...",1.452,[],"[drama, thriller, comedy]","[burglari, lostfilm, neonoir, breakingandent, ...",[christophernolan]
2959,11660,Following,7.151,1412,Following,"[bill, an, idl, unemploy, aspir, writer, walk,...",12.093,"[you, re, never, alon]","[drama, thriller]","[london, england, robberi, stalker, thief, sta...",[christophernolan]
287112,1245917,Crimson Star,0.000,0,Crimson Star,"[plot, under, wrap]",0.750,[],"[sciencefiction, action]",[],[]
1,157336,Interstellar,8.417,32571,Interstellar,"[the, adventur, of, a, group, of, explor, who,...",140.241,"[mankind, was, born, on, earth, it, was, never...","[adventure, drama, sciencefiction]","[rescu, futur, spacecraft, raceagainsttim, art...",[christophernolan]
160014,651692,Virtual Death Match,6.000,2,Virtual Death Match,"[a, group, of, gamer, are, select, to, take, p...",3.470,[],"[horror, action, sciencefiction, adventure]",[],[louisawarren]
114,1124,The Prestige,8.203,14696,The Prestige,"[a, mysteri, stori, of, two, magician, whose, ...",48.832,"[are, you, watch, close]","[drama, mystery, sciencefiction]","[dyinganddeath, suicid, classsocieti, magic, c...",[christophernolan]
41,272,Batman Begins,7.701,19561,Batman Begins,"[driven, by, tragedi, billionair, bruce, wayn,...",66.286,"[evil, fear, the, knight]","[action, crime, drama]","[martialart, undercov, lossoflovedon, secretid...",[christophernolan]


### __Saving the model__

In [37]:
filename_model = '../model/movies_recommendation_engine_model.sav'
joblib.dump(modelKNN, filename_model)

['../model/movies_recommendation_engine_model.sav']

In [45]:
filename_vectorizers= '../model/movies_recommendation_engine_vectorizers.sav'
joblib.dump(vectorizers, filename_vectorizers)


['../model/movies_recommendation_engine_vectorizers.sav']

# __API: API OF SYSTEM RECOMMENDATION ENGINE__

## __THE ML MODEL__

### __Python Imports__

In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack
import joblib

In [34]:
nltk.download('stopwords')
stemmer = SnowballStemmer(language='english')
tokenizer = RegexpTokenizer(r'\w+').tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gabriel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### __UTILS FUNCTIONS__

In [7]:
def clean_str(text: str, lower=True, clean=True) -> str:
    text = text.strip()
    if lower:
        text = text.lower()
    if clean:
        text = re.sub(r'[^a-z\s]'," ", text) if lower else re.sub(r'[^a-zA-Z\s]'," ", text)
    return text

def split_and_clean_str(x: str, delimiter=",", remove_spaces: bool=False, to_lower: bool=True, clean=True):
    new_list = []
    if isinstance(x, str) and x != None and len(x.strip()) > 0 and x !='\\N' and x != np.nan:
        new_list = [clean_str(val, to_lower, clean) for val in x.split(delimiter) if val.strip() != ""]
    if isinstance(x, list):
        new_list = [clean_str(val, to_lower, clean) for val in x if val and val.strip() != ""]
    if remove_spaces:
        new_list = [val.replace(" ", "") for val in new_list]
    return new_list

def word_lists_to_str(word_lists: list, weight) -> str:
    return " ".join([word for word_list in word_lists  
              for word in word_list*weight if word not in stopwords.words('english')])


def split_with_tokenizer(text: str) -> list:
    if isinstance(text, str):
        return tokenizer(text)
    return []

def nplist_to_native_python(nplist: np.array):
    return [x.item() for x in nplist]

def clean_dataframe(dataframe: pd.DataFrame):
    dataframe = dataframe.copy()
    dataframe.fillna({'tagline':'', 'overview': '', 'title': '', 'original_title': '', 'poster_path': '', 'release_date':'', 'vote_average': 0}, inplace=True)
    dataframe['directors'] = dataframe['directors'].apply(lambda x: split_and_clean_str(x, to_lower=False, clean=False))
    dataframe['genres'] = dataframe['genres'].apply(lambda x: split_and_clean_str(x, to_lower=False))    
    dataframe['keywords'] = dataframe['keywords'].apply(split_and_clean_str)
    return dataframe

def process_dataframe(dataframe: pd.DataFrame):
    dataframe = dataframe.copy()
    dataframe['directors'] = dataframe['directors'].apply(lambda x: split_and_clean_str(x, remove_spaces=True))
    dataframe['genres'] = dataframe['genres'].apply(lambda x: split_and_clean_str(x, remove_spaces=True))    
    dataframe['keywords'] = dataframe['keywords'].apply(split_and_clean_str)    
    dataframe['overview'] = dataframe['overview'].apply(split_with_tokenizer)    
    dataframe['tagline'] = dataframe['tagline'].apply(split_with_tokenizer)    
    dataframe['keywords'] = dataframe['keywords'].apply(lambda x: [stemmer.stem(i).replace(" ", "") for i in x])    
    dataframe['tagline'] = dataframe['tagline'].apply(lambda x: [stemmer.stem(i) for i in x])    
    dataframe['overview'] = dataframe['overview'].apply(lambda x: [stemmer.stem(i) for i in x])
    return dataframe
    

### __IMPORT THE ML MODEL AND OTHERS__

#### __Import the dataset__

In [10]:
tmdb_full = pd.read_csv(filepath_or_buffer="../Data_Files/data/TMDB_movies_full.csv", sep=",")

In [11]:
tmdb_full = clean_dataframe(tmdb_full)

#### __Import the Model__

In [13]:
filename_model = '../model/movies_recommendation_engine_model.sav'
model = joblib.load(filename_model)

#### __Import the Vectorizers__

In [15]:
filename_vectorizer = '../model/movies_recommendation_engine_vectorizers.sav'
vectorizers = joblib.load(filename_vectorizer)

### __GETTING THE RECOMENDATIONS__

#### __The recommendation function__

In [18]:
def get_recommendations_engine(dataframe: pd.DataFrame) -> list[int]:
    ids = dataframe['id'].to_list()
    dataframe = process_dataframe(dataframe)
    matrix = []
    
    for col,(vect, weight) in vectorizers.items():
        col_value = [word_lists_to_str(dataframe[col].to_list(), weight)]
        vector = vect.transform(col_value)
        matrix.append(vector)
        
    matrices = hstack(matrix)
    distances, indices = model.kneighbors(matrices, n_neighbors=100)
    
    return clean_dataframe(tmdb_full.loc[indices[0]].loc[~tmdb_full['id'].isin(ids)].sort_values('popularity', ascending=False).head(10)).to_dict('records')

## THE SERVER

In [20]:
#!pip install fastapi

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()

origins = [
    "http://localhost",
    "http://localhost:5173",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class Movie(BaseModel):
    id: int = None
    title: str = ''
    genres: list[str] = []
    directors: list[str] = []
    overview: str = ''
    tagline: str = ''
    keywords: list[str] = []
    release_date: str = ''
    poster_path: str = ''
    vote_average: float = None

@app.get("/")
async def root():
    return {"Message": "This is a recommendation system engine"}

@app.post("/recommendations", response_model=list[Movie])
def get_recommendations(movies: list[Movie]):
    df = pd.DataFrame([movie.dict() for movie in movies])
    movies_dict = get_recommendations_engine(df)
    return movies_dict
    
if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app)